In [ ]:
pip install xmltodict

In [ ]:
import xmltodict
import matplotlib.pyplot as plt
import os

Конвертація датасету IAMONDO з .inkml в .png

In [ ]:
def read_xml(file):
  '''
    Obtaining tracers and matrix values from inkml file

    input :
      file - inkml file opened in read mode
    output :
      tracers - list of tracers from file
      x_vector_float,y_vector_float - vectors of ratios of tracer coordinates to coordinates of points in the image
  '''
  xml_content = file.read()
  data_dict = xmltodict.parse(xml_content)
  tracers = data_dict['ink']['trace']
  try :
    matrix = data_dict['ink']['definitions']['canvasTransform']['mapping']['matrix']
    matrix = matrix.split(',')
    matrix_list = matrix[:2]
    x_vector = matrix_list[0].split()[:2]
    y_vector = matrix_list[1].split()[:2]

    x_vector_float = [float(i) for i in x_vector]
    y_vector_float = [float(i) for i in y_vector]
  except KeyError :
    x_vector_float = [1,0]
    y_vector_float = [0,1]
  return tracers, x_vector_float,y_vector_float

In [ ]:
def tracer_to_list(tracer) :
  '''
  Converting tracer from str to list type
  '''
  elements = tracer.split(',')
  tracer_list = []
  for element in elements:
    clean_elem = element.replace('"', ' ').replace("'", ' ').replace('-',' -').replace(' .',' 0.').replace(' -.',' -0.')
    sublist = clean_elem.split()
    tracer_point = []
    for num in sublist[:2]:
      tracer_point.append(float(num))
    tracer_list.append(tracer_point)
  return tracer_list

In [ ]:
def tracer_coords(tracer_list):
  '''
  Transforming tracer from list type into list of coordinates,
  converting velocities and accelerations to actual coordinates
  '''
  coords_list = []

  start_values = tracer_list[0]
  x0 = start_values[0]
  y0 = start_values[1]
  coords_list.append([x0,y0])

  if len(tracer_list)>1  :
    start_speed = tracer_list[1]
    x_sp = start_speed[0]
    y_sp = start_speed[1]
    x = x0 + x_sp
    y = y0 + y_sp
    coords_list.append([x,y])

    if len(tracer_list)>2 :
      for elems in tracer_list[2:]:
        x_sp_change = elems[0]
        y_sp_change = elems[1]
        x_sp = x_sp + x_sp_change
        y_sp = y_sp + y_sp_change
        x = x + x_sp
        y = y + y_sp
        coords_list.append([x,y])

  return coords_list

In [ ]:
def coords_to_image_lamondo(file,path,name):
  '''
  Drawing and saving an image based on inkml file

  input :
    file - inkml file opened in read mode
    path - path to saving image
    name - the name for the image we'll get
  '''
  tracers, x_vector,y_vector = read_xml(file)
  plt.figure()
  for tracer_dict in tracers:
    tracer = tracer_dict['#text']
    tracer_list = tracer_to_list(tracer)
    coordinates = tracer_coords(tracer_list)

    x_coords = [x_vector[0] * point[0] + x_vector[1] * point[1] for point in coordinates]
    y_coords = [- y_vector[0] * point[0] - y_vector[1] * point[1] for point in coordinates]
    plt.plot(x_coords, y_coords,  color='black')
  plt.axis('off')
  plt.savefig(path+'/'+name+".png", dpi=500, bbox_inches='tight')
  plt.close()

In [ ]:
for root, dirs, files in os.walk("/content/drive/MyDrive/Colab_Notebooks/CV_handwriting/lamono_"):
    for filename in files:
      print(filename)
      with open("/content/drive/MyDrive/Colab_Notebooks/CV_handwriting/lamono_/"+filename, 'r', encoding='utf-8') as file:
          coords_to_image_lamondo(file,"/content/drive/MyDrive/Colab_Notebooks/CV_handwriting/lamono_my_render",filename)


Було : https://drive.google.com/drive/folders/12N_dKOSoV4xXweXLkB6RhtwSb6YNECOy?usp=drive_link

Стало : https://drive.google.com/drive/folders/13z-_YfXSYCrWbUPhlXeCRn7n_wft_klQ?usp=drive_link

Конвертація датасету FA_ з .inkml в .png

In [ ]:
def coords_to_image_fa(file,path,name):
  '''
  Drawing and saving an image based on inkml file

  input :
    file - inkml file opened in read mode
    path - path to saving image
    name - the name for the image we'll get
  '''
  tracers, x_vector,y_vector = read_xml(file)
  plt.figure()
  for tracer_dict in tracers:
    tracer = tracer_dict['#text']
    coordinates = tracer_to_list(tracer)

    x_coords = [x_vector[0] * point[0] + x_vector[1] * point[1] for point in coordinates]
    y_coords = [- y_vector[0] * point[0] - y_vector[1] * point[1] for point in coordinates]

    plt.plot(x_coords, y_coords,  color='black')
  plt.axis('off')
  plt.savefig(path+'/'+name+".png", dpi=500, bbox_inches='tight')
  plt.close()

In [ ]:
for root, dirs, files in os.walk("/content/drive/MyDrive/Colab_Notebooks/CV_handwriting/FA_"):
    for filename in files:
      print(filename)
      with open("/content/drive/MyDrive/Colab_Notebooks/CV_handwriting/FA_/"+filename, 'r', encoding='utf-8') as file:
          coords_to_image_fa(file,"/content/drive/MyDrive/Colab_Notebooks/CV_handwriting/FA_my_render",filename)

Було : https://drive.google.com/drive/folders/1138oRAeXgWEQdeT5Ssy2IIyBE9DjpoGM?usp=drive_link

Стало : https://drive.google.com/drive/folders/1QE8Bb4GIz3GCoQAHSOfGNJTGcYCeVlRh?usp=drive_link